In [1]:
import os
import subprocess
import sys
import ribasim
import ribasim.nodes
import toml
import tomli
import numpy as np
import pandas as pd
import pathlib
import tqdm.auto as tqdm
import warnings
import datetime
from pathlib import Path
import logging

from shapely.geometry import Point

#load ribasim_nl
module_path = Path.cwd() / '../../ribasim_nl/'
sys.path.append(str(module_path))
from ribasim_nl import CloudStorage

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

%reload_ext autoreload
%autoreload 2
warnings.filterwarnings('ignore')
    
import peilbeheerst_model.ribasim_parametrization as ribasim_param
from peilbeheerst_model.ribasim_feedback_processor import RibasimFeedbackProcessor

from peilbeheerst_model.controle_output import *
from ribasim import Solver

from ribasim import Allocation, Model, Node
from ribasim.nodes import (
    basin,
    discrete_control,
    flow_boundary,
    fractional_flow,
    level_boundary,
    level_demand,
    linear_resistance,
    manning_resistance,
    outlet,
    pid_control,
    pump,
    tabulated_rating_curve,
    user_demand,
)

## Define variables and model

#### Set Config

In [2]:
# Set paths
waterschap = "WetterskipFryslan"
versie = "2024_6_17"

work_dir = pathlib.Path(r"../../../../../Ribasim_base_models", waterschap)
ribasim_gpkg = work_dir.joinpath("database.gpkg")
path_ribasim_toml = work_dir.joinpath("ribasim.toml")
output_dir = work_dir.joinpath("results")

# Basin area percentage
regular_percentage = 10
boezem_percentage = 90
unknown_streefpeil = 0.00012345 #we need a streefpeil to create the profiles, Q(h)-relations, and af- and aanslag peil for pumps

# Forcing settings
start_time = "2024-01-01"
timestep_size = 'd'
timesteps = 2


## Process the feedback form

In [3]:
# name = "Ron Bruijns (HKV)"

# feedback_excel = pathlib.Path(r"../../../../../Ribasim_feedback/V1_formulieren/feedback_formulier_" + waterschap + ".xlsx")
# feedback_excel_processed = r"../../../../..//Ribasim_feedback/V1_formulieren_verwerkt/feedback_formulier_" + waterschap + "_JA_processed.xlsx"

# ribasim_toml = pathlib.Path(r"../../../../../Ribasim_base_models", waterschap + '_boezemmodel_' + versie, 'ribasim.toml')
# output_folder = work_dir #r"../../../../../Ribasim_updated_models/AmstelGooienVecht"

# processor = RibasimFeedbackProcessor(name, waterschap, versie, feedback_excel, ribasim_toml, output_folder, feedback_excel_processed)
# processor.run()



In [4]:
# path_ribasim_toml = os.path.join(work_dir, waterschap + '_boezemmodel_' + versie, 'ribasim.toml')
path_ribasim_toml = "../../../../../Ribasim_base_models/WetterskipFryslan_boezemmodel_2024_6_17/ribasim.toml"
output_folder = work_dir #r"../../../../../Ribasim_updated_models/AmstelGooienVecht"


In [6]:
let op! verwijder bovenstaand toml file als er meer feedback komt

SyntaxError: invalid syntax (3021745423.py, line 1)

#### Load model

In [7]:
# Load Ribasim model
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    ribasim_model = ribasim.Model(filepath=path_ribasim_toml)

## Model specific tweaks

In [8]:
assert not pd.isnull(ribasim_model.basin.area.df.meta_streefpeil).any()

ribasim_model.basin.area.df.loc[ribasim_model.basin.area.df['meta_streefpeil'] == 'Onbekend streefpeil', 'meta_streefpeil'] = unknown_streefpeil

ribasim_model.basin.area.df.loc[ribasim_model.basin.area.df['meta_streefpeil'] == 9.999, 'meta_streefpeil'] = unknown_streefpeil

### Add a Terminal and a gemaal at the northernmost Zeepolder

In [9]:
#First, add the gemaal
node_id_gemaal = ribasim_param.get_current_max_nodeid(ribasim_model) + 1 #define new node_id number
ribasim_model.pump.add(Node(node_id_gemaal, Point(170900, 591437)), [pump.Static(flow_rate=[700 / 60])])  #add pump
ribasim_model.edge.add(ribasim_model.basin[328], ribasim_model.pump[node_id_gemaal]) #add the egde from the basin to the pump

node_id_terminal = ribasim_param.get_current_max_nodeid(ribasim_model) + 1 #define new node_id number
ribasim_model.terminal.add(Node(node_id_terminal, Point(170898, 591445))) #add Terminal
ribasim_model.edge.add(ribasim_model.pump[node_id_gemaal], ribasim_model.terminal[node_id_terminal]) #add the edge from the pump to the terminal

#add the edges between the two nodes above



In [10]:
# These nodes have suddenly been added, without apparent reason. Delete these nodes
ribasim_model.pump.node.df = ribasim_model.pump.node.df.loc[ribasim_model.pump.node.df.node_id != 94374]
ribasim_model.pump.node.df = ribasim_model.pump.node.df.loc[ribasim_model.pump.node.df.node_id != 94376]


# Parameterization

## Nodes

### Basin (characteristics)

In [11]:
# Define the initial state of each basin. Is set equal to the streefpeil
# ribasim_param.set_initial_basin_state(ribasim_model) #the initial states of the basins are by default already set to the streefpeil!

# Insert standard profiles to each basin. These are [depth_profiles] meter deep, defined from the streefpeil
ribasim_param.insert_standard_profile(ribasim_model, 
                                      regular_percentage = regular_percentage, 
                                      boezem_percentage = boezem_percentage,
                                      depth_profile = 2)

In [ ]:
# Add storage basins
# model_name = 'AmstelGooienVecht_StorageBasins'
# node_ids = [1, 2, 3]  # Specify node IDs to process or include_hoofdwater Boolean
processor = AddStorageBasin(ribasim_toml, model_name, output_folder, include_hoofdwater=True, log=True, node_ids=None)
ribasim_model = processor.run()

# Check basin area
ribasim_param.validate_basin_area(ribasim_model)

### Basin (forcing)

In [12]:
# Set static forcing
forcing_dict = {'precipitation':         ribasim_param.convert_mm_day_to_m_sec(5),
                'potential_evaporation': ribasim_param.convert_mm_day_to_m_sec(0),
                'drainage':              ribasim_param.convert_mm_day_to_m_sec(0),
                'infiltration':          ribasim_param.convert_mm_day_to_m_sec(0),
                'urban_runoff':          ribasim_param.convert_mm_day_to_m_sec(0),
                }

ribasim_param.set_static_forcing(timesteps, timestep_size, start_time, forcing_dict, ribasim_model)


### Pumps

In [13]:
# Set pump capacity for each pump
ribasim_model.pump.static.df['flow_rate'] = 0.16667 # 10 kuub per minuut

### FlowBoundary

In [14]:
# Set FlowBoundary rate for each pump
ribasim_model.flow_boundary.static.df['flow_rate'] = 0 #

### Add Discrete Control

In [15]:
# Add discrete control nodes and control edges
ribasim_param.add_discrete_control_nodes(ribasim_model)

### TabulatedRatingCurve

In [16]:
# First, set the Q(h)-relationship to a relationship which is at each TRC node the same
ribasim_param.set_tabulated_rating_curves(ribasim_model, level_increase=1.0, flow_rate=1)

# The water level in the boezem is regulated heavily. There is a smaller tolerance for an increase in water level. 
# Hence, the TRC's which discharge water from the boezem to a Terminal should have a higher capacity.
ribasim_param.set_tabulated_rating_curves_boundaries(ribasim_model, level_increase=0.10, flow_rate=40)


# Set numerical settings

In [17]:
# Write model output
ribasim_param.index_reset(ribasim_model)
ribasim_model.starttime = datetime.datetime(2024, 1, 1)
ribasim_model.endtime = datetime.datetime(2024, 1, 1, 0, 0, 1)
ribasim_model.solver.saveat = 3600
ribasim_param.write_ribasim_model_Zdrive(ribasim_model, 
                                         path_ribasim_toml)

## Run Model

## Iterate over tabulated rating curves

In [18]:
try:
    ribasim_param.iterate_TRC(ribasim_param = ribasim_param, 
                          allowed_tolerance = 0.02, 
                          max_iter = 1, 
                          expected_difference = 0.1, 
                          max_adjustment = 0.25, 
                          cmd = ['ribasim', path_ribasim_toml], 
                          output_dir = output_dir, 
                          path_ribasim_toml = path_ribasim_toml)

except Exception as e:
    logging.error("The model was not able to run. Log file:")
    log_file_path = os.path.join(output_dir, 'ribasim.log') 
    try:
        with open(log_file_path, 'r') as log_file:
            log_content = log_file.read()
            print(log_content)
    except Exception as log_exception:
        logging.error(f"Could not read the log file: {log_exception}")

Simulating init:   0%|          | 0/100 [00:00<?, ?it/s]

ERROR:root:The model was not able to run. Log file:
ERROR:root:Could not read the log file: [Errno 2] No such file or directory: '../../../../../Ribasim_base_models/WetterskipFryslan/results/ribasim.log'


# Write model

In [22]:
path_ribasim_toml = "../../../../../Ribasim_updated_models/WetterskipFryslan/ribasim.toml"

In [21]:
pas wederom het pad hierboven aan

SyntaxError: invalid syntax (715709383.py, line 1)

In [23]:
# control_dict = Control(work_dir = work_dir).run_all()
ribasim_param.write_ribasim_model_GoodCloud(ribasim_model = ribasim_model, 
                                            path_ribasim_toml = path_ribasim_toml,
                                            waterschap = waterschap, 
                                            modeltype = 'boezemmodel',
                                            include_results = True)


The model of waterboard WetterskipFryslan has been uploaded to the goodcloud in the directory of boezemmodel!


## Open Output

In [20]:
df_basin = pd.read_feather(os.path.join(output_dir, 'basin.arrow'))
df_basin

FileNotFoundError: [Errno 2] No such file or directory: '../../../../../Ribasim_base_models/WetterskipFryslan/results/basin.arrow'